In [1]:
import os
import pandas as pd

### Step 1
`src.prompt_creator.create_prompt` モジュールを使用して、実験に使用するプロンプトを生成する。

In [ ]:
from src.prompt_creator.create_prompt import PromptCreator

In [3]:
prompt_dir = "data/prompt_output"

In [ ]:
promptcreator = PromptCreator("NLI", "SA", 100)

for method in ["combined-attack", "context-ignoring", "escape-characters", "fake-completion", "naive-attack"]:
    os.makedirs(os.path.join(prompt_dir, method), exist_ok=True)
    prompt= promptcreator.create_prompt(method)
    prompt_df = pd.DataFrame(prompt)
    prompt_df.to_csv(os.path.join(prompt_dir, method, f"NLI_SA_100.csv"), index=False)

### Step 2
`src.llm_processor.llm_processor` モジュールを使用して、LLMに回答を出力させる

In [ ]:
from src.llm_processor.llm_processor import LLM

In [3]:
llm_input_dir = "data/prompt_output"
llm_output_dir = "data/llm_output"

In [ ]:
for model in ["gemini-1.5-flash", "gemini-1.5-pro"]:
    llm = LLM(provider="gemini", model=model)
    for method in ["combined-attack", "context-ignoring", "escape-characters", "fake-completion", "naive-attack"]:
        print(f"Processing {model} {method}")
        os.makedirs(os.path.join(llm_output_dir, model, method), exist_ok=True)
        prompt = pd.read_csv(os.path.join(llm_input_dir, method, "NLI_SA_100.csv"))
        llm_output = llm.create_llm_outputs(prompt)
        llm_output.to_csv(os.path.join(llm_output_dir, model, method, f"NLI_SA_100.csv"), index=False)

### Step 3
`src.evaluator.attack_benchmark` を用いて攻撃の評価をする

In [2]:
from src.evaluator.attack_benchmark import (
    asv,
    mr,
    pna
)

In [3]:
llm_output_dir = "data/llm_output"
results_dir = "output"

In [ ]:
for model in ["gemini-1.5-flash", "gemini-1.5-pro"]:
    for method in ["combined-attack", "context-ignoring", "escape-characters", "fake-completion", "naive-attack"]:
        os.makedirs(os.path.join(results_dir, model, method), exist_ok=True)
        print(f"Evaluating {model} {method}")
        
        llm_output_df = pd.read_csv(os.path.join(llm_output_dir, model, method, "NLI_SA_100.csv"))
        output_df = pd.DataFrame()
        output_df["ASV"] = asv.asv(llm_output_df)["ASV"]
        output_df["MR"] = mr.mr(llm_output_df)["MR"]
        pna_df = pna.pna(llm_output_df)
        output_df["PNA-T"] = pna_df["PNA-T"]
        output_df["PNA-I"] = pna_df["PNA-I"]
        
        output_df.to_csv(os.path.join(results_dir, model, method, "NLI_SA_100.csv"), index=False)